In [71]:
from langchain.agents import Tool, initialize_agent, AgentType
from langchain_openai import ChatOpenAI 
from langchain.memory import ConversationBufferMemory
import os
from dotenv import load_dotenv
import requests
from datetime import datetime
load_dotenv()

True

In [72]:
# Define the Email Management Tool
def manage_email(action: str, email_details: dict):
    # Implement email management logic (sending, reading, etc.)
    return f"Email action '{action}' executed with details: {email_details}."

# Define the Calendar Scheduling Tool
def schedule_event(event_details: dict):
    # Implement event scheduling logic
    try:  
        return f"Event scheduled on {event_details['date']}."
    except Exception as e:
        return "Shain"
 
 # Define the Social Media Management Tool
def manage_social_media(action: str, content: str):
    # Implement social media management logic
    return f"Social media action '{action}' executed with content: {content}."   
   
def fetch_holidays(country_code="US"):
    api_key = os.getenv("HOLIDAY_API_KEY")  # Ensure your API key is set in .env
    current_year = datetime.now().year  # Get the current year
    url = f"https://calendarific.com/api/v2/holidays?api_key={api_key}&country=BD&year={current_year}"

    response = requests.get(url)

    # Debugging: Print the raw response
    #print(f"Raw Response Content: {response.content}")
    #print(f"Response Status Code: {response.status_code}")

    try:
        response_data = response.json()  # Parse response to JSON
        #print(f"Parsed JSON: {response_data}")  # Debugging: Check the parsed JSON
    except ValueError as ve:
        #print(f"Error: Unable to parse the response as JSON. Details: {ve}")
        return "Error: Unable to parse the response as JSON."

    # Safely get the 'holidays' key from the JSON
    holidays = response_data.get('response', {}).get('holidays', [])

    # Debugging: Check if holidays are found
    #print(f"Holidays Data: {holidays}")

    if holidays:
        # Create a simple summary of the holidays
        holiday_summary = "\n".join([f"{holiday['name']} on {holiday['date']['iso']}" for holiday in holidays])
        return f"Holidays in {country_code} for {current_year}:\n{holiday_summary}"
    else:
        return "No holidays found or invalid response."

In [73]:

email_management_tool = Tool(
    name="EmailManagement",
    func=manage_email,
    description="Manage your emails (send, read, delete)."
)

calendar_tool = Tool(
    name="CalendarScheduling",
    func=schedule_event,
    description="Schedule calendar events."
)

social_media_tool = Tool(
    name="SocialMediaManagement",
    func=manage_social_media,
    description="Manage your social media accounts."
)

holiday_tool = Tool(
    name="HolidayService",
    func=fetch_holidays,
    description="Fetch information about holidays for a given country."
)
# List of tools
tools = [
    email_management_tool,
    calendar_tool,
    social_media_tool,
    holiday_tool
    # Add other tools here...
]


In [74]:
# Initialize your LLM
llm = ChatOpenAI(temperature=0,api_key=os.getenv("OPEN_AI_KEY"),model_name="gpt-4")

# Initialize memory
memory = ConversationBufferMemory()

# Initialize the agent
agent = initialize_agent(
    tools,
    llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=False
)

In [75]:
# Function to handle user queries
def handle_user_query(query):
    try:
        response = agent.invoke(query)
    except Exception as e:
        # Fallback or handle the exception
        response = "I'm sorry, I couldn't complete that request."
    return response["output"]

# Example usage
if __name__ == "__main__":
    # User queries
    try:
        user_query_1 = "How many holydays in this next month of this year from now and give me the list of these?"
        response_1 = handle_user_query(user_query_1)
        print(response_1)
    except Exception as e:
        print(e)



The holidays for the next month are: 
1. Ash Wednesday on 2024-02-14
2. Valentine's Day on 2024-02-14
3. Language Martyrs' Day on 2024-02-21
4. Maghi Purnima on 2024-02-23
5. Shab e-Barat on 2024-02-26
